In [10]:
import os
import concurrent.futures
import pandas as pd
import geopandas as gpd
import requests
from tqdm import tqdm
from PIL import Image
from sklearn.cluster import MeanShift
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
from skimage.color import rgb2hsv
from skimage.filters.rank import modal

In [11]:
# 画像の保存先ディレクトリ
IMAGE_DIR = "shibuya_images"

In [12]:
points = gpd.read_file("/workspace/app/Green-View-Index/points_shibuya/shibuya_points_200.shp")
max_id = points["point_id"].max()
headings = [0,60,120,180,240,300]
pitch = 0

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, segmentation, color, filters


def graythresh(array, level):
    """OTSUのしきい値を計算"""
    maxVal = np.max(array)
    minVal = np.min(array)

    if maxVal <= 1:
        array = array * 255
    elif maxVal >= 256:
        array = np.int((array - minVal) / (maxVal - minVal))

    negIdx = np.where(array < 0)
    array[negIdx] = 0

    hist = np.histogram(array, range(257))
    P_hist = hist[0] * 1.0 / np.sum(hist[0])

    omega = P_hist.cumsum()
    temp = np.arange(256)
    mu = P_hist * (temp + 1)
    mu = mu.cumsum()

    n = len(mu)
    mu_t = mu[n - 1]
    sigma_b_squared = (mu_t * omega - mu) ** 2 / (omega * (1 - omega))
    maxval = np.max(sigma_b_squared)

    index = np.where(sigma_b_squared == maxval)
    idx = np.mean(index)
    threshold = (idx - 1) / 255.0

    if np.isnan(threshold):
        threshold = level

    return threshold


In [14]:
def extract_greenery(image):
    """緑の領域を抽出してマスクを作成"""

    # Quickshiftでセグメンテーション
    segments = segmentation.quickshift(image, kernel_size=3, max_dist=6, ratio=0.5)

    # 画像の正規化
    I = image / 255.0

    # RGBチャンネルの取得
    red = I[:, :, 0]
    green = I[:, :, 1]
    blue = I[:, :, 2]

    # 緑の領域を強調
    green_red_diff = green - red
    green_blue_diff = green - blue
    ExG = green_red_diff + green_blue_diff

    # Otsuのしきい値を使用して緑のピクセルを抽出
    threshold = graythresh(ExG, 0.1)
    green_mask = ExG > threshold

    return green_mask



In [15]:
def calculate_gvi(green_mask):
    """GVIを計算する関数"""

    # 緑のピクセル数をカウント
    green_pixel_count = np.sum(green_mask)

    # 画像全体のピクセル数を取得
    total_pixel_count = green_mask.size

    # GVIの計算（緑のピクセルの割合）
    gvi = (green_pixel_count / total_pixel_count) * 100  # パーセンテージ表記

    return gvi

In [16]:
from tqdm import tqdm

# tqdmを使用して進捗バーを表示
for idx, point in tqdm(points.iterrows(), total=len(points), desc="Processing GVI for points"):
    point_id = point["point_id"]
    gvi = 0
    for heading in headings:
        # 画像の読み込み
        if not os.path.exists(f"{IMAGE_DIR}/{point_id}_{heading}_{pitch}.jpg"):
            continue
        image = io.imread(f"{IMAGE_DIR}/{point_id}_{heading}_{pitch}.jpg")
        # 緑のマスクを取得
        green_mask = extract_greenery(image)
        # GVIを計算
        gvi += calculate_gvi(green_mask)

    # 各方向のGVIの平均を取る
    gvi /= len(headings)

    # GVIをデータフレームに保存
    points.loc[idx, "gvi"] = gvi

Processing GVI for points:   0%|          | 0/3102 [00:00<?, ?it/s]

Processing GVI for points:   0%|          | 0/3102 [01:43<?, ?it/s]


KeyboardInterrupt: 